In [1]:
! pip install -qU langchain_ollama

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
loader = CSVLoader(file_path="recipies_add_dif-241019.csv", encoding='utf-8')
data = loader.load()
print(f"Loaded {len(data)} documents from the csv.")

Loaded 5612 documents from the csv.


In [5]:
import os

current_path = os.getcwd()
print("Current path:", current_path)

# colab의 보안 비밀에 OPENAI_API_KEY를 등록해서 환경변수로 설정
#from google.colab import userdata
#
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Current path: /Users/khuh/AI-Training/LLM-RAG


In [6]:
#위에서 불러온 csv 데이터 청크
#RecursiveCharacterTextSplitter를 사용
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(data)
len(splits)

5977

In [7]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=ollama_emb)

# 백터스토어를 로컬에 저장합니다.
# 7분 27.8초 소요
vectorstore.save_local('./db/faiss5-ollama-mxbai')

# 벡터스토리에서 검색하는 retriever를 만듭니다.
retriever = vectorstore.as_retriever()
# 소요시간 9m 14.6s

In [ ]:
### 벡터스토어 불러오기
#vectorstore = FAISS.load_local('./db/faiss5-ollama-mxbai', ollama_emb, allow_dangerous_deserialization=True)
#retriever = vectorstore.as_retriever()

In [8]:
### 백터스토어 작동 테스트
# 검색 예시
query = "토마토"
results = retriever.get_relevant_documents(query)

# 검색 결과 출력
for result in results:
    print(result.page_content)


url: https://www.10000recipe.com//recipe/6880048
reason: 이 요리는 간단하고 쉽게 만들 수 있는 나물 반찬입니다. 조리 시간이 15분이며, 재료의 양과 조리 방법이 적절히 설명되어 있습니다. 또한, 각 단계별로 구체적인 설명이 제공되어 초기부터 마지막까지 쉽게 이해할 수 있도록 돕습니다. 그러나 무침의 양념 부분에서 소금과 참기름 등의 양을 조절하는 것이 필요하다는 점을 명시해야 하는 등 조금 더 구체적인 조리 지침이 필요할 수 있습니다.
difficulties: medium
﻿title: 티타임에 어울리는 디저트  홍차 향을 담은 얼그레이 스콘만들기
servings: 4 servings
cooking_time: PT30H-1710M
ingredients: 박력분 220g,베이킹파우더 8g,버터 50g,설탕 50g,소금 1꼬집,생크림 70g,우유 50g,홍차티백 2개,달걀물 약간
cooking_orders: ['1. 볼에 홍차티백을 뜯어서 넣고 뜨거운 물 1큰술을 넣고 불린다.', '2. 설탕, 생크림, 우유, 홍차 불린 것을 넣고 섞는다.', '3. 푸드프로세서에 체친 가루재료, 소금, 차가운 버터를 넣고 돌린다.', '4. 가루가 보슬보슬하게 되면 볼에 옮긴 후 2를 넣고 스크래퍼로 섞는다.', '5. 한덩어리가 되면 랩으로 싸서 냉장고에서 1시간 이상 휴지시킨다.', '6. 휴지시킨 반죽을 2cm 두께로 밀어편 뒤 원하는 모양으로 자른다.', '7. 오븐팬에 반죽을 올리고 윗면에 달걀물을 발라준다.', '8. 220도로 예열된 오븐에 7분 굽고 200도로 내려7분 굽고 식힘망에 옮긴다.', '9. 구워진 스콘이 따뜻할 때 잼이나 버터를 곁들어 완성한다.']
url: https://www.10000recipe.com//recipe/6932456
reason: 조리 시간이 PT30H-1710M(약 3일 17시간 10분)로 매우 길며, 조리 단계가 9단계로 복잡하고, 여러 가지 재료와 단계가 복잡하게 섞여 있어 상급

/var/folders/py/mwpg_rm904g6ld02xtcmncwh0000gn/T/ipykernel_12703/101046714.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [10]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
    당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
    검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
    요리를 추천할 때는 3개의 가장 유사한 요리를 요리 제목과 함께 추천해주세요.
    요리 방법을 답할때는 재료를 목록으로 요리 순서를 목록으로 출력하고 출처도 함께 표시해줘.
    만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
    항상 한국어로 답변해주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [11]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="qwen2.5:3b",
)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# ChatGPT model 사용
#llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인을 생성합니다.
#rag_chain = (
#    {"context": retriever, "question": RunnablePassthrough()}
#    | prompt
#    | llm
#    | StrOutputParser()
#)

In [12]:
# 
question = "30분 이내에 만들 수 있는 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

According to the difficulty levels provided for each recipe, here's an analysis of which one might be considered more challenging:

1. 신림 순대촌이 우리 집에 볶는 이 음식은 모든 재료를 조리해야 하므로 여러 단계가 필요합니다. 특히 양배추와 깻잎을 추가로 볶아야 해서, 중급자 수준으로 보입니다.

2. 빗물만들기에서 조리 시간이 30분으로 적절하지만, 다양한 재료를 준비하고 여러 단계의 조리를 필요로 하므로, 중급자 수준으로 판단합니다.

3. 김치찌개는 조리 과정이 명확하게 기재되어 있지만, 양배추와 깻잎을 추가로 볶아야 하는 점이 복잡할 수 있으므로, 중급자 수준으로 보입니다.

4. 빽순대볶음은 여러 가지 재료를 준비하고 조리해야 하며, 특히 부추와 깻잎, 들기름 등을 추가로 볶아서 양배추의 본래 맛을 살리려면 많은 노력이 필요합니다. 중급자 수준으로 판단됩니다.

5. 밥을 넣어 볶는 음식은 조리 시간이 30분이고 재료의 종류와 양이 많지 않아 상급자에게 어려움이 없을 것으로 보입니다. 그러나 여러 가지 빵과 다양한 소스를 준비하고 조리해야 하므로, 중급자 수준으로 보는 게 적절할 것입니다.

6-8번은 각각의 재료 볶기만 해도 복잡한 조리 과정이 요구되는 것으로 보입니다. 이런 점을 고려하면, 이 두 메뉴는 상급자에게 어려움이 있을 수 있습니다.

따라서 가장 어렵게 평가될 수 있는 메뉴는 신림 순대촌이 우리 집에 볶는 음식과 빽순대볶음입니다. 양배추와 깻잎을 추가로 조리해야 하거나 여러 가지 재료를 동시에 준비하고 조리는 중급자 이상의 기술과 시간을 필요로 합니다.


In [13]:
question = "토마토를 이용한 간단한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

Based on the cooking difficulties described in the provided recipes, here's an analysis of which recipe might be considered more challenging:

Recipe 1 (참외 피클):
- It involves multiple steps like washing and preparing various ingredients, thin slicing them, soaking with salt water, then steaming or simmering.
- The final combination also includes making a spicy dipping sauce. Overall complexity is moderate to high due to the numerous steps and types of preparation needed.

Recipe 2 (비올 때 생각나는 수제비):
- While it has several stages like mixing dough, cooling, slicing vegetables, cooking in broth, and finishing with seasoning.
- The difficulty seems lower when compared to Recipe 1. It requires basic skills for mixing, simmering soup, and preparing small quantities of ingredients.

Considering these points, the "참외 피클" recipe (Recipe 2) might be considered more challenging due to its comprehensive preparation steps and complex final combination.


In [14]:
question = "두부를 이용한 요리를 하나 추천하고 조리 방법을 알려주고 출처도 표시해줘"
answer = rag_chain.invoke(question)
print(answer)

Let's analyze the difficulties level of each recipe based on their ingredients and cooking steps. The main criteria for determining difficulty are the number of ingredients used, the complexity of mixing, cooking time, and the need for multiple steps.

1. **기름이 부드럽게 올라가도록 재우고 청경채를 넣어 볶아요.**
   - Difficulty: *초*
   
2. **양념장과 양파를 볶아서 소스로 만들고, 그 후 양념장과 같이 볶아서 단백질 요리에 활용합니다.**
   - Difficulty: *간*

3. **대형 양배추가 구워지면 만들어둔 양념장을 넣어 볶아요.**
   - Difficulty: *중*

4. **생강, 채소, 조미료를 모두 함께 볶는다.**
   - Difficulty: *간*

5. **대형 양배추가 구워지면 만들어둔 양념장을 넣어 볶아요.**
   - Difficulty: *중*

6. **채소가 구워지면 만들어둔 양념장을 넣어 볶아요.**
   - Difficulty: *간*

7. **양념과 잘 섞어 꿀 또는 알룰로스를 넣어 볶아요.**
   - Difficulty: *중*

8. **청경채를 넣어 빠르게 볶아 불을 끄고 참기름을 둘러 매콤허니두부조림을 완성해요.**
   - Difficulty: *간*

9. **현미밥 위에 올려서 후추와 깨소금을 뿌려 완성해요.**
   - Difficulty: *초*

10. **강력분, 물, 소금 등을 넣고 고속으로 반죽한다.**
    - Difficulty: *중*

Based on this analysis, the difficulty levels are as follows:
- 4 and 5 are classified as *중* (medium) because they involve 

In [15]:
question = "짜장면 만드는 방법 알려줘"
answer = rag_chain.invoke(question)
print(answer)

Based on the difficulties provided for each recipe, I would rank these recipes from easiest to hardest as follows:

1. "당근요리" (Easy): This recipe only requires a few ingredients and has straightforward instructions that are easy to follow.
2. "다이어트는 곤약밥으로 시작  냉동도시락 7일치 만들기" (Medium-Hard): Although it involves multiple steps including preparation of different ingredients, the overall difficulty is higher due to its longer cooking time and more complex procedures.
3. "당 싸요리" (Hard): This recipe has a high difficulty level mainly because of its numerous steps, diverse ingredients requiring various preparations, and extensive final mixing process.

Therefore, the ranking from easiest to hardest would be:
1. 당근요리
2. 다이어트는 곤약밥으로 시작  냉동도시락 7일치 만들기
3. 당 싸요리


In [16]:
question = "백종원 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

Based on the provided document, here are the steps to make "This Young's Fish Pancake" (called 이영자의 붕어빵 in Korean):

1. **Prepare Ingredients**: Prepare 1 cucumber, 1 shiitake mushroom, and half a white part of ginger, as well as 100g pork belly meat for stewing, one tablespoon of vinegar, a little amount of sesame oil, a little amount of green onion, soy sauce, sesame seeds.

2. **Stew Pork Belly**: 
   - Mix all ingredients in a bowl and let it sit.
   - Heat the wok (or pan) and add some sesame oil.
   - Stir-fry the pork belly until browned.
   
3. **Prepare Cucumber, Shiitake Mushrooms, and Green Onion**:
   - Cut cucumber into thin slices.
   - Clean shiitake mushrooms thoroughly.
   - Clean green onions and cut them diagonally.

4. **Combine**: 
   - Add stewed pork belly to the wok with sliced cucumber, chopped mushrooms, and sliced green onions.
   - Stir-fry until everything is heated evenly and add sesame oil last for a nice aroma and flavor.

5. **Add Sesame Seeds**:
   - P